# Импорты

In [ ]:
import numpy as np
import pandas as pd
from statistics import mode
from multiprocessing import Pool
from sklearn.base import BaseEstimator


def entropy(y):
    p = [len(y[y == k]) / len(y) for k in np.unique(y)]
    return -np.dot(p, np.log2(p))


def gini(y):
    p = [len(y[y == k]) / len(y) for k in np.unique(y)]
    return 1 - np.dot(p, p)


def variance(y):
    return np.var(y)


def mad_median(y):
    return np.mean(np.abs(y - np.median(y)))


def regression_leaf(y):
    return np.mean(y)


def classification_leaf(y):
    return mode(y)


class Node:
    def __init__(self, feature_idx=0, threshold=0, labels=None, left=None, right=None):
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.labels = labels
        self.left = left
        self.right = right


class DecisionTree(BaseEstimator):
    def __init__(self, max_depth=np.inf, min_samples_split=2, min_samples_leaf=1, criterion="variance",
                 leaf_func="regression_leaf"):
        params = {
            "max_depth": max_depth,
            "min_samples_split": min_samples_split,
            "min_samples_leaf": min_samples_leaf,
            "criterion": criterion,
            "leaf_func": leaf_func
        }

        criteria_dict = {
            "variance": variance,
            "mad_median": mad_median,
            "gini": gini,
            "entropy": entropy
        }

        leaf_dict = {
            "regression_leaf": regression_leaf,
            "classification_leaf": classification_leaf
        }

        for param_name, param_value in params.items():
            setattr(self, param_name, param_value)

        super(DecisionTree, self).set_params(**params)
        self._criterion_function = criteria_dict[criterion]
        self._leaf_value = leaf_dict[leaf_func]
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None
        self.current_depth = 0

    def _functional(self, x_train: pd.DataFrame, y: pd.Series, feature_idx: int, threshold):
        mask = x_train.iloc[:, feature_idx] < threshold
        n_obj = x_train.shape[0]
        n_left = np.sum(mask)
        n_right = n_obj - n_left
        if n_left > 0 and n_right > 0:
            return (
                    self._criterion_function(y)
                    - (n_left / n_obj) * self._criterion_function(y.loc[mask])
                    - (n_right / n_obj) * self._criterion_function(y.loc[~mask])
            )
        else:
            return 0

    def _build_tree(self, x_train: pd.DataFrame, y: pd.Series, depth=1):
        """Train decision tree"""
        max_functional = 0
        best_feature_idx = None
        best_threshold = None
        n_samples, n_features = x_train.shape

        if len(np.unique(y)) == 1:
            return Node(labels=y)

        best_mask = None
        if depth < self.max_depth and n_samples >= self.min_samples_split and n_samples >= self.min_samples_leaf:
            for feature_idx in range(n_features):
                max_value = np.max(x_train.iloc[:, feature_idx])
                min_value = np.min(x_train.iloc[:, feature_idx])
                threshold_values = np.linspace(min_value, max_value, 5)
                functional_values = [
                    self._functional(x_train, y, feature_idx, threshold) for threshold in threshold_values
                ]

                best_threshold_idx = np.nanargmax(functional_values)

                if functional_values[best_threshold_idx] > max_functional:
                    max_functional = functional_values[best_threshold_idx]
                    best_threshold = threshold_values[best_threshold_idx]
                    best_feature_idx = feature_idx
                    best_mask = x_train.iloc[:, feature_idx] < best_threshold

        if best_feature_idx is not None and best_mask is not None:
            return Node(
                feature_idx=best_feature_idx,
                threshold=best_threshold,
                left=self._build_tree(x_train.loc[best_mask], y.loc[best_mask], depth + 1),
                right=self._build_tree(x_train.loc[~best_mask, :], y.loc[~best_mask], depth + 1),
            )
        else:
            self.current_depth = depth
            return Node(labels=y)

    def fit(self, x_train: pd.DataFrame, y: pd.Series):
        """Run training decision tree"""
        self.root = self._build_tree(x_train, y)
        self.max_depth = self.current_depth
        return self

    def _predict_object(self, x: pd.Series):
        """Prediction for one test object"""
        node = self.root
        while node.labels is None:
            if x[node.feature_idx] < node.threshold:
                node = node.left
            else:
                node = node.right
        return self._leaf_value(node.labels)

    def predict(self, x_test: pd.DataFrame) -> np.array:
        """Prediction for all test objects"""
        results = np.array([self._predict_object(x_test.iloc[i]) for i in range(0, x_test.shape[0])])
        return np.array(results)


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.base import BaseEstimator


class GradientBoosting(BaseEstimator):
    def __init__(self, n_estimators=10, learning_rate=0.01, max_depth=3, min_samples_split=5, criterion="variance",
                 leaf_func="regression_leaf", random_state=17, loss_name="mse"):

        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.leaf_func = leaf_func
        self.random_state = random_state
        self.learning_rate = learning_rate
        self.loss_name = loss_name
        self.initialization = lambda y: np.mean(y) * np.ones([y.shape[0], 1])

        if loss_name == "mse":
            self.objective = self.mse
            self.objective_grad = self.mse_grad

        elif loss_name == "rmsle":
            self.objective = self.rmsle
            self.objective_grad = self.rmsle_grad

        self.trees_ = []

    @staticmethod
    def mse(y, p):
        return np.mean((y - p) ** 2)

    @staticmethod
    def mse_grad(y: np.array, p: np.array):
        return 2 * (p - y) / y.shape[0]

    @staticmethod
    def rmsle(y, p):
        y = y.reshape([y.shape[0], 1])
        p = p.reshape([p.shape[0], 1])
        return np.mean(np.log((p + 1) / (y + 1)) ** 2) ** 0.5

    def rmsle_grad(self, y, p):
        y = y.reshape([y.shape[0], 1])
        p = p.reshape([p.shape[0], 1])
        return 1.0 / (y.shape[0] * (p + 1) * self.rmsle(y, p)) * np.log((p + 1) / (y + 1))

    def fit(self, X: np.array, y: np.array):
        b = self.initialization(y)
        prediction = b.copy()

        for t in tqdm(range(self.n_estimators)):
            if t == 0:
                resid = y
            else:
                resid = -self.objective_grad(y, prediction)

            tree = DecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                criterion=self.criterion,
                leaf_func=self.leaf_func
            )
            tree.fit(X, pd.Series(resid))
            b = tree.predict(X).reshape([X.shape[0], 1])
            self.trees_.append(tree)
            """
                вот тут можно найти такой learning_rate, что функция потерь от таргета и пресдказания*learning_rate
                будет минимальна но я просто умножила его на предсказания и сложила с прошлыми предсказаниями
                (обновляем текущее предсказание)
            """
            prediction += self.learning_rate * b
        return self

    def predict(self, X):
        predictions = np.ones([X.shape[0], 1])
        for t in range(self.n_estimators):
            predictions += self.learning_rate * self.trees_[t].predict(X).reshape([X.shape[0], 1])
        return predictions

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn import svm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, roc_curve, r2_score, mean_squared_error

# Функции для отрисовки

In [ ]:
def show_histplot(data: pd.DataFrame, feature_name: str):
    sns.histplot(data, kde=True, binwidth=0.1)
    plt.xlabel(f'Значения {feature_name}')
    plt.ylabel('Частота')
    plt.title(f'Распределение {feature_name}')
    plt.show()


def get_boxplot(df_column, column_name):
    pd.DataFrame(df_column).boxplot(sym='o', whis=1.0, showmeans=True)
    plt.show()


def get_3d(param1: list[int], param2: list[int], result: list[int], name_param1: str, name_param2: str):
    fig = plt.figure()
    ax = plt.axes(projection ='3d')
    ax.plot3D(param1, param2, result, 'green')
    ax.set_title(f'Зависимость метрики R² от {name_param1} и {name_param2}')
    plt.show()

def get_2d(param1: list[int], result: list[int], name_param1: str):
    plt.title(f'Зависимость метрики R² от {name_param1}')
    plt.plot(param1, result)

# Всякие полезные функции

In [ ]:
# Evaluation function
def evaluation(model_name, y_test, y_pred_test, output=False):
    r2_test = r2_score(y_test, y_pred_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

    if output:
        print(f'\n{model_name} Evaluation:')
        print(f'Test R²: {r2_test:.7f}')
        print(f'Test RMSE: {rmse_test:.5f}')
    return r2_test

def output_metrics_classification(y_test: pd.Series, preds: pd.Series):
    report = classification_report(y_test, preds, target_names=['Non-fail', 'Fail'])
    print(report)

def output_roc_auc(y_test: pd.Series, preds: pd.Series):
    sns.set(font_scale=1.5)
    sns.set_color_codes("muted")

    plt.figure(figsize=(10, 8))
    fpr, tpr, thresholds = roc_curve(y_test, preds, pos_label=1)
    lw = 2
    plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
    plt.plot([0, 1], [0, 1])
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.savefig("ROC.png")
    plt.show()

# EDA: Marketing campaign

In [ ]:
train_flood = pd.read_csv('train.csv')
train_flood = train_flood.drop(['id'], axis=1)
train_flood

## Распределение признаков

In [ ]:
[show_histplot(train_flood[column], column) for column in train_flood.columns]

## Тепловая карта

In [ ]:
sns.heatmap(train_flood.corr(), vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')

## Выбросы

In [ ]:
[get_boxplot(train_flood[column], column) for column in train_flood.columns]

#### Смотрим на выбросы в процентах

In [ ]:
def find_outliers(df):
    outliers = {}
    for col in df.columns:
        v = df[col]
        q1 = v.quantile(0.25)
        q3 = v.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers_count = ((v < lower_bound) | (v > upper_bound)).sum()
        perc = outliers_count * 100.0 / len(df)
        outliers[col] = (perc, outliers_count)
        print(f"Column {col} outliers = {perc:.2f}%")

    return outliers

outliers = find_outliers(train_flood)

**Итог по выбросам:**
- как будто бы до пизды на выбросы, тем более не везде они выбросы

## Feature Engineering

In [ ]:
train_flood['mean'] = train_flood[train_flood.columns].mean(axis=1)
train_flood['std'] = train_flood[train_flood.columns].std(axis=1)
train_flood['max'] = train_flood[train_flood.columns].max(axis=1)
train_flood['median'] = train_flood[train_flood.columns].median(axis=1)

*ИТОГ*:

Я решила ввести новые признаки, потому что в древовидных моделях (Дерево решений, случайный лес, градиентный бустинг) нет естественного способа агрегирования информации по многим объектам одновременно.

## Смотрим на важность признаков и на зависимость

In [ ]:
X = train_flood
y = X['FloodProbability']
X = X.drop('FloodProbability', axis=1, inplace=False)

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

discrete_features = X.dtypes == int
mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]
plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

# Разделение на train/test/valid

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

# Обучение/тест - Дерево решений

## Моя реализация

In [ ]:
min_samples_split = [4, 10, 15]
min_samples_leaf = [3, 9, 14]
best_tree = None
best_metric = -1

for leaf in min_samples_leaf:
    for slpit in min_samples_split:
        tree = DecisionTree(max_depth=100, min_samples_split=slpit, min_samples_leaf=leaf, criterion="variance", leaf_func="regression_leaf")
        tree.fit(X_train, y_train)
        predictions = tree.predict(X_test)

        r2 = evaluation("Decision Tree", y_test, predictions, output=False)
        print(f'Обучили дерево с параметрами min_samples_leaf={leaf}, min_samples_split={slpit}, depth={tree.max_depth}: R² = {r2}')
        if best_metric < r2:
            best_metric = r2
            best_tree = tree

*ИТОГ*:

Есть некое наитие, что дерево с максимальной глубиной равной 15 переобучилось.

In [ ]:
max_depth = range(5,12)
metric_values = []
for depth in max_depth:
    tree = DecisionTree(max_depth=depth, min_samples_split=5, criterion="variance", leaf_func="regression_leaf")
    tree.fit(X_train, y_train)
    predictions = tree.predict(X_test)

    r2 = evaluation("Decision Tree", y_test, predictions, output=False)
    print(f'Обучили дерево с параметром max_depth={depth}: R² = {r2}')
    metric_values.append(r2)

In [ ]:
get_2d(max_depth, metric_values, "max_depth")

### Проверяем, есть ли переобучение

Давайте протестируем на тестовой выборке деревья с наилучшими результатами.

P.S.
Тут я использую X_valid, y_valid, но на самом деле из-за того, как определила валидационную и тестовую выборку, без разници, какую использовать для валидации, а какую для теста. Мне просто лень исправлять и перезапускать ячейки.

In [ ]:
tree = DecisionTree(max_depth=15, min_samples_split=15, criterion="variance", leaf_func="regression_leaf")
tree.fit(X_train, y_train)
predictions = tree.predict(X_val)

r2 = evaluation("Decision Tree", y_val, predictions, output=True)

**ИТОГ:**

Тестирование показало, что всё ок, но я бы всё не брала дерево с такой глубиной.

## Библиотичная реализация

In [ ]:
X = train_flood
y = X['FloodProbability']
X = X.drop('FloodProbability', axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lib_tree = DecisionTreeRegressor()
param_grid = {
    'max_depth': [5, 7, 11],
    'min_samples_split': [4, 6, 10]
}

grid_search = GridSearchCV(lib_tree, param_grid, cv=4, scoring='r2', verbose=True)
grid_search.fit(X, y)
print("Лучшие параметры:", grid_search.best_params_)
print("Лучший счет:", grid_search.best_score_)

**ИТОГ:**

Моя реализация дала лучшие результаты при тех же параметрах, но думаю, что на самом деле это не совсем так. В реализации sklearn используется такая вещь как Minimal Cost-Complexity Pruning, которое предотвращает переобучение.

# Обучение/тест - Случайный лес

## Моя реализация

### Найдем лучшие параметры

In [ ]:
max_depth = [5, 7, 11]
n_estimators = [5, 15, 25]

best_metric = -1
best_forest = None

for depth in max_depth:
    for estimators in n_estimators:
        forest = RandomForest(max_depth=depth, min_samples_split=10, min_samples_leaf=11, criterion="variance", leaf_func = "regression_leaf", N=estimators)
        forest.fit(X_train, y_train)
        preds_forest = forest.predict(X_test)
        r2 = evaluation("Random Forest", y_test, preds_forest, output=False)

        if r2 > best_metric:
            best_forest = forest
            best_metric = r2

In [ ]:
print(f"Параметры лучшего леса: количество деревьев={best_forest.N}, глубина={best_forest.max_depth}, r2={best_metric}")

### Посмотрим теперь на зависимость метрики от количества деревьев

In [ ]:
n_estimators = range(10,25,2)
metrics = []

for estimators in n_estimators:
    forest = RandomForest(max_depth=11, min_samples_split=10, min_samples_leaf=11, criterion="variance", leaf_func = "regression_leaf", N=estimators)
    forest.fit(X_train, y_train)
    preds_forest = forest.predict(X_test)
    r2 = evaluation("Random Forest", y_test, preds_forest, output=False)
    metrics.append(r2)


In [ ]:
get_2d(n_estimators, metrics, "n_estimators")

## Библиотечная реализация

### Найдем лучшие параметры

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lib_forest = RandomForestRegressor()
param_grid = {
    'max_depth': [5, 7, 11],
    'n_estimators': [5, 10, 15]
}

grid_search = GridSearchCV(lib_forest, param_grid, cv=4, scoring='r2', verbose=True)
grid_search.fit(X, y)
print("Лучшие параметры:", grid_search.best_params_)
print("Лучший счет:", grid_search.best_score_)

### Посмотрим теперь на зависимость метрики от количества деревьев

In [ ]:
lib_forest = RandomForestRegressor()
param_grid = {
    'n_estimators': range(10, 25, 2)
}

grid_search = GridSearchCV(lib_forest, param_grid, cv=2, scoring='r2', verbose=True)
grid_search.fit(X, y)

In [ ]:
get_2d(n_estimators, grid_search.cv_results_['mean_test_score'], "n_estimators")

# Обучение/тест - Градиентный бустинг

## Моя реализация

### Подберем лучшие параметры

In [ ]:
max_depth = [5, 7, 11]
n_estimators = [5, 15, 25]

best_metric = -1
best_forest = None

for depth in max_depth:
    for estimators in n_estimators:
        boosting = GradientBoosting()
        boosting.fit(X_train, y_train)
        preds_boosting = forest.predict(X_test)
        r2 = evaluation("Gradient boosting", y_test, preds_boosting, output=False)

        if r2 > best_metric:
            best_forest = forest
            best_metric = r2

In [ ]:
print(f"Параметры лучшего леса: количество деревьев={best_forest.N}, глубина={best_forest.max_depth}, r2={best_metric}")

### Посмотрим на зависимость между количеством деревьев и метрикой

In [ ]:
n_estimators = range(10,25,2)
metrics = []

for estimators in n_estimators:
    boosting = GradientBoosting()
    boosting.fit(X_train, y_train)
    preds_boosting = forest.predict(X_test)
    r2 = evaluation("Gradient boosting", y_test, preds_boosting, output=False)
    metrics.append(r2)